In [1]:
import numpy as np 
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
import pandas as pd
import collections
from random import shuffle
import itertools
from tqdm import tqdm

In [2]:
tmp = collections.defaultdict(set)

In [37]:
np.array(np.array([1,2,3,4,5,6]) == np.array([1,2,3,9,3,5]),dtype = 'i4')

array([1, 1, 1, 0, 0, 0])

In [3]:
data_set = np.load('user_movie_rating.npy')

In [4]:
df = pd.DataFrame(data_set)

In [5]:
df.rename(columns=  {0:'user_id',1:'movie_id',2:'rating'},inplace = True)

columns -> users

row -> movies 

We permutate the rows and get their signature which is the signature of the colmuns then -> users that's what we need then

In [6]:
%%time
person_u = list(np.sort(df.user_id.unique()).squeeze())
thing_u = list(np.sort(df.movie_id.unique()).squeeze())

data = df['rating'].astype('bool').astype(int).tolist()
# row = person_u
# col = thing_u
# col
col = df.user_id.astype('category').cat.codes
row = df.movie_id.astype('category').cat.codes
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(thing_u),len(person_u)))

CPU times: total: 6.69 s
Wall time: 12.1 s


In [7]:
# sparse_matrix[2]

In [8]:
tmp = sparse_matrix.getrow(2).todense()

In [9]:
sparse_matrix

<17770x103703 sparse matrix of type '<class 'numpy.intc'>'
	with 65225506 stored elements in Compressed Sparse Row format>

In [130]:
%%time
indices = np.arange(sparse_matrix.shape[0]) #gets the number of rows 
shuffle(indices)
shuffled_matrix = sparse_matrix[list(indices)] 
arr = np.array(shuffled_matrix.argmax(axis=0))[0,:]
sign_matrix = arr
for i in range(149):
    # indices = np.arange(sparse_matrix.shape[0]) #gets the number of rows 
    shuffle(indices)
    shuffled_matrix = sparse_matrix[list(indices)] 
    arr = np.array(shuffled_matrix.argmax(axis=0))[0,:]
    sign_matrix = np.vstack([sign_matrix,arr])

CPU times: total: 2min 27s
Wall time: 3min 20s


In [101]:
sign_matrix.shape

(100, 103703)

In [102]:
sign_matrix[:10]

array([[ 80,  31,  80, ...,  80, 167,  80],
       [ 56,  27,  17, ..., 125,   7, 125],
       [  8,  21,  13, ...,   6,  21,   8],
       ...,
       [  6,   6,   6, ...,  14,   6,  68],
       [ 13,  58,  11, ...,  13,  66,  13],
       [  3,  14,   3, ...,  37,   0,  26]])

In [103]:
def hashing_mod(sign_matrix,buckets,rows,band):
    

IndentationError: expected an indented block (1060968691.py, line 2)

In [20]:
def fastCandidatePairs(sig_mat, b, r):
    n, d = sig_mat.shape
    # assert(n==b*r)
    hashbuckets = collections.defaultdict(set)
    bands = np.array_split(sig_mat, b, axis=0)
    for i,band in enumerate(bands):
        if i %4 == 0 :
            print("band: " +str(i))
        for j in range(d):
            # The last value must be made a string, to prevent accidental
            # key collisions of r+1 integers when we really only want
            # keys of r integers plus a band index
            band_id = tuple(list(band[:,j])+[str(i)])
            hashbuckets[band_id].add(j)
    candidate_pairs = set()
    # for bucket in hashbuckets.values():
    #     if len(bucket) > 1:
    #         for pair in itertools.combinations(bucket, 2):
    #             candidate_pairs.add(pair)
    return hashbuckets.values()

In [145]:
%%time
tmp = fastCandidatePairs(sign_matrix,15,10)

band: 0
band: 4
band: 8
band: 12
CPU times: total: 1.94 s
Wall time: 4.03 s


In [ ]:
t = 0
found_pairs = []
buckets = list(tmp)
buckets.sort(key=len)
# print(buckets)
# print('tego')
for bucket in tqdm(buckets):
    # print(len(bucket))
    t+=1
    if len(bucket) >= 2 :
        # print(bucket)
        for pair in itertools.combinations(bucket, 2):
                sim = jacc_sim_base(sparse_matrix, pair[0], pair[1])
                if sim > 0.5:
                    print(pair)
                    found_pairs.append(pair)

 99%|███████████████████████████████████████████████████████████████████████▌| 1524414/1533311 [03:39<03:24, 43.47it/s]

(51236, 52773)


100%|███████████████████████████████████████████████████████████████████████▋| 1526498/1533311 [06:19<08:34, 13.25it/s]

(52494, 99903)


100%|███████████████████████████████████████████████████████████████████████▋| 1527023/1533311 [06:59<08:00, 13.08it/s]

...

100%|███████████████████████████████████████████████████████████████████████▉| 1532948/1533311 [36:40<14:29,  2.39s/it]

(50435, 47929)
(20676, 47929)


100%|████████████████████████████████████████████████████████████████████▉| 1533281/1533311 [13:51:05<43:32, 87.09s/it]

(36382, 47194)


100%|███████████████████████████████████████████████████████████████████▉| 1533289/1533311 [14:05:12<40:52, 111.45s/it]

In [137]:
a,b = np.random.randint(0,103703,2)

63825

In [140]:
t = 0
found_pairs = []
buckets = list(tmp)
buckets.sort(key=len)
# print(buckets)
# print('tego')
for i in tqdm(range(1000000)):
    a,b = np.random.randint(0,103703,2)
    sim = jacc_sim_base(sparse_matrix, a, b)
    if sim > 0.5:
            print(pair)
            found_pairs.append(pair)

  1%|▌                                                                       | 7245/1000000 [09:49<22:25:34, 12.30it/s]


KeyboardInterrupt: 

In [70]:
def jacc_sim_base(matrix,id_1,id_2):
    user_1 = np.where(matrix.getcol(id_1).todense()==1)[0]
    user_2 = np.where(matrix.getcol(id_2).todense()==1)[0]

    all = np.union1d(user_1,user_2)
    inter = np.intersect1d(user_1,user_2)
    # print(np.sum(same_movies))
    # print(same_movies.shape[0])
    
    return inter.shape[0]/all.shape[0]

In [78]:
from A2_sim import jaccard_similarity

Jaccard Similarity: 0.5
Cosine Similarity: 0.6399999999999999
Discrete Cosine Similarity: 0.6666666666666667


In [93]:
jacc_sim_base(sparse_matrix, 59763, 28867)

0.4078091106290672

In [80]:
jaccard_similarity(sparse_matrix.getcol(87017).todense(),sparse_matrix.getcol(68).todense())

0.2593068035943517

In [47]:
b = 10
n, d = sign_matrix.shape
# assert(n==b*r)
hashbuckets = collections.defaultdict(set)
bands = np.array_split(sign_matrix, b, axis=0)

In [48]:
sign_matrix.shape

(81, 103703)

In [49]:
bands[1].shape

(8, 103703)

(103703,)